# Predict a US presidential election using Bayesian optimal polling

In this tutorial, we explore the use of optimal experimental design techniques to create an optimal polling strategy to predict the outcome of the US presidential election. We take the 2012 election as our prior and the 2016 election as our test set: we imagine that we are conducting polling just before the 2016 election.

## The model
For each of the 50 states we define 

$$ \text{logit }\mathbb{P}(\text{a random voter in state } i \text{ votes Democrat in the 2016 election}) = \theta_i $$

and we assume all other voters vote Republican. Right before the election, the value of $\theta$ is unknown and we wish to estimate it using polling. The winner $w$ of the election is decided by the Electoral College system. The number of electoral college votes gained by the Democrats in state $i$ is
$$
e_i =  \begin{cases}
k_i \text{ if } \theta_i > \frac{1}{2} \\
0 \text{ otherwise}
\end{cases}
$$
(this is a rough approximation of the true system). All other electoral college votes go to the Republicans. Here $k_i$ is the number of electoral college votes alloted to state $i$, which are given by

In [80]:
import torch
electoral_college_votes = torch.tensor([9, 8, 3, 3, 8, 18, 11, 4, 7, 6, 10, 7, 55, 11, 20, 9,
                                        16, 4, 7, 10, 9, 3, 6, 3, 3, 10, 11, 29, 3, 38, 16, 5, 
                                        6, 4, 6, 3, 4, 4, 13, 20, 14, 12, 11, 5, 5, 6, 29, 10,
                                        6, 15, 3], dtype=torch.float)
print("50 states plus DC:", len(electoral_college_votes))
print("Total electoral college votes:", sum(electoral_college_votes).item())

50 states plus DC: 51
Total electoral college votes: 538.0


(states are ordered alphabetically)

The winner $w$ is

$$ w = \begin{cases}
D \text{ if } \sum_i e_i > \frac{1}{2}\sum_i k_i  \\
R \text{ otherwise}
\end{cases}
$$

We are interested in polling strategies that will help us predict $w$, rather than predicting the more complex state-by-state results $\theta$.

To set up a fully Bayesian model, we need a prior for $\theta$. For the mean, we will use the outcome of the 2012 election. We could, of course, use some publically available polling data, newspaper articles, etc. to inform the prior but in this notebook we simplify things by assuming ignorance of this kind of information.

In [38]:
p = torch.tensor([0.38783771466562295, 0.4268470951843963, 0.4538661977629681, 0.3784559464877038, 0.6187281116329663, 0.5274798831420252, 0.587725774769071, 0.5944695491927816, 0.9258764920211752, 0.5044225213936623, 0.46043350901943275, 0.7170384850107442, 0.3357861315642467, 0.585775233921169, 0.447996254026727, 0.5295935192810017, 0.38886673254746945, 0.38457227606195094, 0.4125317438588602, 0.5785992139357711, 0.6332172578690598, 0.6178568074906001, 0.5480053207117788, 0.5394122646169819, 0.4419810080121222, 0.45221333246620027, 0.42965768136118543, 0.38870609731774636, 0.5340754216334653, 0.528337519782581, 0.5898685113730959, 0.5529520464690869, 0.6430089453069677, 0.48965965096995, 0.3988210485958018, 0.5151455253695202, 0.3322768025698344, 0.5627116717874738, 0.5273193400189182, 0.6401674597888212, 0.44691745295434626, 0.4078150086180476, 0.3964892847370595, 0.41992102784893615, 0.2537381110485252, 0.6824725882639062, 0.5196737668876448, 0.5762829827062185, 0.36325702367957263, 0.5351638240433735, 0.2883936598847857])
mu = torch.log(p/(1-p))
print(mu)

[8]
tensor([-0.4564, -0.2947, -0.1851, -0.4961,  0.4842,  0.1100,  0.3546,  0.3825,
         2.5250,  0.0177, -0.1586,  0.9298, -0.6821,  0.3465, -0.2088,  0.1185,
        -0.4521, -0.4702, -0.3535,  0.3170,  0.5460,  0.4805,  0.1926,  0.1580,
        -0.2331, -0.1917, -0.2832, -0.4528,  0.1365,  0.1135,  0.3634,  0.2126,
         0.5884, -0.0414, -0.4104,  0.0606, -0.6979,  0.2522,  0.1094,  0.5761,
        -0.2131, -0.3730, -0.4201, -0.3231, -1.0788,  0.7652,  0.0787,  0.3075,
        -0.5613,  0.1409, -0.9032])


Our prior distribution for $\theta$ will be a multivariate Normal with mean `mu`. The only thing left to decide upon is the covariance matrix. The prior covariance is important in a number of ways. If we allow too much variance, the prior will be uncertain about the outcome in every state, and require polling everywhere. If we allow too little variance, we may be caught off-guard by an unexpected electoral outcome. If we assume states are independent, then we will not be able to pool information across states; but assume too much correlation and we could too faithfully base predictions about one state from poll results in another.

We select the prior covariance by taking the empirical covariance from the last five presential elections (1996 - 2012) and adding a small value `0.01` to the diagonal.

In [72]:
ss = [
    """Alabama 	9 	813,479 	38.74% 	- 	1,266,546 	60.32% 	9 	6,788 	0.32% 	- 	4,991 	0.24% 	- 	4,310 	0.21% 	- 	0 	0.00% 	- 	3,705 	0.18% 	- 	-453,067 	-21.58% 	2,099,819 	AL
Alaska 	3 	123,594 	37.89% 	- 	193,841 	59.42% 	3 	3,783 	1.16% 	- 	1,589 	0.49% 	- 	1,660 	0.51% 	- 	0 	0.00% 	- 	1,730 	0.53% 	- 	-70,247 	-21.54% 	326,197 	AK
Arizona 	10 	1,034,707 	45.12% 	- 	1,230,111 	53.64% 	10 	11,301 	0.49% 	- 	12,555 	0.55% 	- 	1,371 	0.06% 	- 	3,406 	0.15% 	- 	24 	0.00% 	- 	-195,404 	-8.52% 	2,293,475 	AZ
Arkansas 	6 	422,310 	38.86% 	- 	638,017 	58.72% 	6 	12,882 	1.19% 	- 	4,776 	0.44% 	- 	4,023 	0.37% 	- 	3,470 	0.32% 	- 	1,139 	0.10% 	- 	-215,707 	-19.85% 	1,086,617 	AR
California 	55 	8,274,473 	61.01% 	55 	5,011,781 	36.95% 	- 	108,381 	0.80% 	- 	67,582 	0.50% 	- 	3,145 	0.02% 	- 	38,774 	0.29% 	- 	57,764 	0.43% 	- 	3,262,692 	24.06% 	13,561,900 	CA
Colorado 	9 	1,288,633 	53.66% 	9 	1,073,629 	44.71% 	- 	13,352 	0.56% 	- 	10,898 	0.45% 	- 	6,233 	0.26% 	- 	2,822 	0.12% 	- 	5,895 	0.25% 	- 	215,004 	8.95% 	2,401,462 	CO
Connecticut 	7 	997,772 	60.59% 	7 	629,428 	38.22% 	- 	19,162 	1.16% 	- 	0 	0.00% 	- 	311 	0.02% 	- 	90 	0.01% 	- 	34 	0.00% 	- 	368,344 	22.37% 	1,646,797 	CT
Delaware 	3 	255,459 	61.94% 	3 	152,374 	36.95% 	- 	2,401 	0.58% 	- 	1,109 	0.27% 	- 	626 	0.15% 	- 	385 	0.09% 	- 	58 	0.01% 	- 	103,085 	25.00% 	412,412 	DE
District of Columbia 	3 	245,800 	92.46% 	3 	17,367 	6.53% 	- 	958 	0.36% 	- 	0 	0.00% 	- 	0 	0.00% 	- 	590 	0.22% 	- 	1,138 	0.43% 	- 	228,433 	85.92% 	265,853 	DC
Florida 	27 	4,282,074 	51.03% 	27 	4,045,624 	48.22% 	- 	28,124 	0.34% 	- 	17,218 	0.21% 	- 	7,915 	0.09% 	- 	2,887 	0.03% 	- 	6,902 	0.08% 	- 	236,450 	2.82% 	8,390,744 	FL
Georgia 	15 	1,844,123 	46.99% 	- 	2,048,759 	52.20% 	15 	1,158 	0.03% 	- 	28,731 	0.73% 	- 	1,402 	0.04% 	- 	250 	0.01% 	- 	63 	0.00% 	- 	-204,636 	-5.21% 	3,924,486 	GA
Hawaii 	4 	325,871 	71.85% 	4 	120,566 	26.58% 	- 	3,825 	0.84% 	- 	1,314 	0.29% 	- 	1,013 	0.22% 	- 	979 	0.22% 	- 	0 	0.00% 	- 	205,305 	45.26% 	453,568 	HI
Idaho 	4 	236,440 	36.09% 	- 	403,012 	61.52% 	4 	7,175 	1.10% 	- 	3,658 	0.56% 	- 	4,747 	0.72% 	- 	39 	0.01% 	- 	51 	0.01% 	- 	-166,572 	-25.43% 	655,122 	ID
Illinois 	21 	3,419,348 	61.92% 	21 	2,031,179 	36.78% 	- 	30,948 	0.56% 	- 	19,642 	0.36% 	- 	8,256 	0.15% 	- 	11,838 	0.21% 	- 	1,160 	0.02% 	- 	1,388,169 	25.14% 	5,522,371 	IL
Indiana 	11 	1,374,039 	49.95% 	11 	1,345,648 	48.91% 	- 	909 	0.03% 	- 	29,257 	1.06% 	- 	1,024 	0.04% 	- 	87 	0.00% 	- 	90 	0.00% 	- 	28,391 	1.03% 	2,751,054 	IN
Iowa 	7 	828,940 	53.93% 	7 	682,379 	44.39% 	- 	8,014 	0.52% 	- 	4,590 	0.30% 	- 	4,445 	0.29% 	- 	1,423 	0.09% 	- 	7,332 	0.48% 	- 	146,561 	9.53% 	1,537,123 	IA
Kansas 	6 	514,765 	41.65% 	- 	699,655 	56.61% 	6 	10,527 	0.85% 	- 	6,706 	0.54% 	- 	4,148 	0.34% 	- 	35 	0.00% 	- 	36 	0.00% 	- 	-184,890 	-14.96% 	1,235,872 	KS
Kentucky 	8 	751,985 	41.17% 	- 	1,048,462 	57.40% 	8 	15,378 	0.84% 	- 	5,989 	0.33% 	- 	4,694 	0.26% 	- 	0 	0.00% 	- 	112 	0.01% 	- 	-296,477 	-16.23% 	1,826,620 	KY
Louisiana 	9 	782,989 	39.93% 	- 	1,148,275 	58.56% 	9 	6,997 	0.36% 	- 	0 	0.00% 	- 	2,581 	0.13% 	- 	9,187 	0.47% 	- 	10,732 	0.55% 	- 	-365,286 	-18.63% 	1,960,761 	LA
Maine★ 	4 	421,923 	57.71% 	4 	295,273 	40.38% 	- 	10,636 	1.45% 	- 	251 	0.03% 	- 	177 	0.02% 	- 	2,900 	0.40% 	- 	3 	0.00% 	- 	126,650 	17.32% 	731,163 	ME
Maryland 	10 	1,629,467 	61.92% 	10 	959,862 	36.47% 	- 	14,713 	0.56% 	- 	9,842 	0.37% 	- 	3,760 	0.14% 	- 	4,747 	0.18% 	- 	9,205 	0.35% 	- 	669,605 	25.44% 	2,631,596 	MD
Massachusetts 	12 	1,904,097 	61.80% 	12 	1,108,854 	35.99% 	- 	28,841 	0.94% 	- 	13,189 	0.43% 	- 	4,971 	0.16% 	- 	6,550 	0.21% 	- 	14,483 	0.47% 	- 	795,243 	25.81% 	3,080,985 	MA
Michigan 	17 	2,872,579 	57.43% 	17 	2,048,639 	40.96% 	- 	33,085 	0.66% 	- 	23,716 	0.47% 	- 	14,685 	0.29% 	- 	8,892 	0.18% 	- 	170 	0.00% 	- 	823,940 	16.47% 	5,001,766 	MI
Minnesota 	10 	1,573,354 	54.06% 	10 	1,275,409 	43.82% 	- 	30,152 	1.04% 	- 	9,174 	0.32% 	- 	6,787 	0.23% 	- 	5,174 	0.18% 	- 	10,319 	0.35% 	- 	297,945 	10.24% 	2,910,369 	MN
Mississippi 	6 	554,662 	43.00% 	- 	724,597 	56.18% 	6 	4,011 	0.31% 	- 	2,529 	0.20% 	- 	2,551 	0.20% 	- 	1,034 	0.08% 	- 	481 	0.04% 	- 	-169,935 	-13.17% 	1,289,865 	MS
Missouri 	11 	1,441,911 	49.29% 	- 	1,445,814 	49.43% 	11 	17,813 	0.61% 	- 	11,386 	0.39% 	- 	8,201 	0.28% 	- 	80 	0.00% 	- 	0 	0.00% 	- 	-3,903 	-0.13% 	2,925,205 	MO
Montana 	3 	231,667 	47.25% 	- 	242,763 	49.51% 	3 	3,686 	0.75% 	- 	1,355 	0.28% 	- 	143 	0.03% 	- 	23 	0.00% 	- 	10,665 	2.18% 	- 	-11,096 	-2.26% 	490,302 	MT
Nebraska★ 	5 	333,319 	41.60% 	1 	452,979 	56.53% 	4 	5,406 	0.67% 	- 	2,740 	0.34% 	- 	2,972 	0.37% 	- 	1,028 	0.13% 	- 	2,837 	0.35% 	- 	-119,660 	-14.93% 	801,281 	NE
Nevada 	5 	533,736 	55.15% 	5 	412,827 	42.65% 	- 	6,150 	0.64% 	- 	4,263 	0.44% 	- 	3,194 	0.33% 	- 	1,411 	0.15% 	- 	6,267 	0.65% 	- 	120,909 	12.49% 	967,848 	NV
New Hampshire 	4 	384,826 	54.13% 	4 	316,534 	44.52% 	- 	3,503 	0.49% 	- 	2,217 	0.31% 	- 	226 	0.03% 	- 	40 	0.01% 	- 	3,624 	0.51% 	- 	68,292 	9.61% 	710,970 	NH
New Jersey 	15 	2,215,422 	57.27% 	15 	1,613,207 	41.70% 	- 	21,298 	0.55% 	- 	8,441 	0.22% 	- 	3,956 	0.10% 	- 	3,636 	0.09% 	- 	2,277 	0.06% 	- 	602,215 	15.57% 	3,868,237 	NJ
New Mexico 	5 	472,422 	56.91% 	5 	346,832 	41.78% 	- 	5,327 	0.64% 	- 	2,428 	0.29% 	- 	1,597 	0.19% 	- 	1,552 	0.19% 	- 	0 	0.00% 	- 	125,590 	15.13% 	830,158 	NM
New York 	31 	4,804,945 	62.88% 	31 	2,752,771 	36.03% 	- 	41,249 	0.54% 	- 	19,596 	0.26% 	- 	634 	0.01% 	- 	12,801 	0.17% 	- 	8,935 	0.12% 	- 	2,052,174 	26.86% 	7,640,931 	NY
North Carolina 	15 	2,142,651 	49.70% 	15 	2,128,474 	49.38% 	- 	1,448 	0.03% 	- 	25,722 	0.60% 	- 	0 	0.00% 	- 	158 	0.00% 	- 	12,336 	0.29% 	- 	14,177 	0.33% 	4,310,789 	NC
North Dakota 	3 	141,278 	44.62% 	- 	168,601 	53.25% 	3 	4,189 	1.32% 	- 	1,354 	0.43% 	- 	1,199 	0.38% 	- 	0 	0.00% 	- 	0 	0.00% 	- 	-27,323 	-8.63% 	316,621 	ND
Ohio 	20 	2,940,044 	51.50% 	20 	2,677,820 	46.91% 	- 	42,337 	0.74% 	- 	19,917 	0.35% 	- 	12,565 	0.22% 	- 	8,518 	0.15% 	- 	7,149 	0.13% 	- 	262,224 	4.59% 	5,708,350 	OH
Oklahoma 	7 	502,496 	34.35% 	- 	960,165 	65.65% 	7 	0 	0.00% 	- 	0 	0.00% 	- 	0 	0.00% 	- 	0 	0.00% 	- 	0 	0.00% 	- 	-457,669 	-31.29% 	1,462,661 	OK
Oregon 	7 	1,037,291 	56.75% 	7 	738,475 	40.40% 	- 	18,614 	1.02% 	- 	7,635 	0.42% 	- 	7,693 	0.42% 	- 	4,543 	0.25% 	- 	13,613 	0.74% 	- 	298,816 	16.35% 	1,827,864 	OR
Pennsylvania 	21 	3,276,363 	54.49% 	21 	2,655,885 	44.17% 	- 	42,977 	0.71% 	- 	19,912 	0.33% 	- 	1,092 	0.02% 	- 	0 	0.00% 	- 	17,043 	0.28% 	- 	620,478 	10.32% 	6,013,272 	PA
Rhode Island 	4 	296,571 	62.86% 	4 	165,391 	35.06% 	- 	4,829 	1.02% 	- 	1,382 	0.29% 	- 	675 	0.14% 	- 	797 	0.17% 	- 	2,121 	0.45% 	- 	131,180 	27.81% 	471,766 	RI
South Carolina 	8 	862,449 	44.90% 	- 	1,034,896 	53.87% 	8 	5,053 	0.26% 	- 	7,283 	0.38% 	- 	6,827 	0.36% 	- 	4,461 	0.23% 	- 	0 	0.00% 	- 	-172,447 	-8.98% 	1,920,969 	SC
South Dakota 	3 	170,924 	44.75% 	- 	203,054 	53.16% 	3 	4,267 	1.12% 	- 	1,835 	0.48% 	- 	1,895 	0.50% 	- 	0 	0.00% 	- 	0 	0.00% 	- 	-32,130 	-8.41% 	381,975 	SD
Tennessee 	11 	1,087,437 	41.83% 	- 	1,479,178 	56.90% 	11 	11,560 	0.44% 	- 	8,547 	0.33% 	- 	8,191 	0.32% 	- 	2,499 	0.10% 	- 	2,337 	0.09% 	- 	-391,741 	-15.07% 	2,599,749 	TN
Texas 	34 	3,528,633 	43.68% 	- 	4,479,328 	55.45% 	34 	5,751 	0.07% 	- 	56,116 	0.69% 	- 	5,708 	0.07% 	- 	909 	0.01% 	- 	1,350 	0.02% 	- 	-950,695 	-11.77% 	8,077,795 	TX
Utah 	5 	327,670 	34.41% 	- 	596,030 	62.58% 	5 	8,416 	0.88% 	- 	6,966 	0.73% 	- 	12,012 	1.26% 	- 	982 	0.10% 	- 	294 	0.03% 	- 	-268,360 	-28.18% 	952,370 	UT
Vermont 	3 	219,262 	67.46% 	3 	98,974 	30.45% 	- 	3,339 	1.03% 	- 	1,067 	0.33% 	- 	500 	0.15% 	- 	66 	0.02% 	- 	1,838 	0.57% 	- 	120,288 	37.01% 	325,046 	VT
Virginia 	13 	1,959,532 	52.63% 	13 	1,725,005 	46.33% 	- 	11,483 	0.31% 	- 	11,067 	0.30% 	- 	7,474 	0.20% 	- 	2,344 	0.06% 	- 	6,355 	0.17% 	- 	234,527 	6.30% 	3,723,260 	VA
Washington 	11 	1,750,848 	57.65% 	11 	1,229,216 	40.48% 	- 	29,489 	0.97% 	- 	12,728 	0.42% 	- 	9,432 	0.31% 	- 	3,819 	0.13% 	- 	1,346 	0.04% 	- 	521,632 	17.18% 	3,036,878 	WA
West Virginia 	5 	303,857 	42.59% 	- 	397,466 	55.71% 	5 	7,219 	1.01% 	- 	0 	0.00% 	- 	2,465 	0.35% 	- 	2,355 	0.33% 	- 	89 	0.01% 	- 	-93,609 	-13.12% 	713,451 	WV
Wisconsin 	10 	1,677,211 	56.22% 	10 	1,262,393 	42.31% 	- 	17,605 	0.59% 	- 	8,858 	0.30% 	- 	5,072 	0.17% 	- 	4,216 	0.14% 	- 	8,062 	0.27% 	- 	414,818 	13.90% 	2,983,417 	WI
Wyoming 	3 	82,868 	32.54% 	- 	164,958 	64.78% 	3 	2,525 	0.99% 	- 	1,594 	0.63% 	- 	1,192 	0.47% 	- 	0 	0.00% 	- 	1,521 	0.60% 	- 	-82,090 	-32.24% 	254,658 	WY""",
    """Alabama 	9 	1,176,394 	62.46% 	9 	693,933 	36.84% 	– 	6,701 	0.36% 	– 	3,529 	0.19% 	– 	1,994 	0.11% 	– 	0 	0.00% 	– 	898 	0.05% 	– 	482,461 	25.62% 	1,883,449 	AL
Alaska 	3 	190,889 	61.07% 	3 	111,025 	35.52% 	– 	5,069 	1.62% 	– 	1,675 	0.54% 	– 	2,092 	0.67% 	– 	1,058 	0.34% 	– 	790 	0.25% 	– 	79,864 	25.55% 	312,598 	AK
Arizona 	10 	1,104,294 	54.87% 	10 	893,524 	44.40% 	– 	2,773 	0.14% 	– 	11,856 	0.59% 	– 	0 	0.00% 	– 	138 	0.01% 	– 	0 	0.00% 	– 	210,770 	10.47% 	2,012,585 	AZ
Arkansas 	6 	572,898 	54.31% 	6 	469,953 	44.55% 	– 	6,171 	0.58% 	– 	2,352 	0.22% 	– 	2,083 	0.20% 	– 	1,488 	0.14% 	– 	0 	0.00% 	– 	102,945 	9.76% 	1,054,945 	AR
California 	55 	5,509,826 	44.36% 	– 	6,745,485 	54.31% 	55 	20,714 	0.17% 	– 	50,165 	0.40% 	– 	26,645 	0.21% 	– 	40,771 	0.33% 	– 	27,747 	0.22% 	– 	−1,235,659 	−9.95% 	12,421,353 	CA
Colorado 	9 	1,101,255 	51.69% 	9 	1,001,732 	47.02% 	– 	12,718 	0.60% 	– 	7,664 	0.36% 	– 	2,562 	0.12% 	– 	1,591 	0.07% 	– 	2,808 	0.13% 	– 	99,523 	4.67% 	2,130,330 	CO
Connecticut 	7 	693,826 	43.95% 	– 	857,488 	54.31% 	7 	12,969 	0.82% 	– 	3,367 	0.21% 	– 	1,543 	0.10% 	– 	9,564 	0.61% 	– 	12 	0.00% 	– 	−163,662 	−10.37% 	1,578,769 	CT
Delaware 	3 	171,660 	45.75% 	– 	200,152 	53.35% 	3 	2,153 	0.57% 	– 	586 	0.16% 	– 	289 	0.08% 	– 	250 	0.07% 	– 	100 	0.03% 	– 	−28,492 	−7.59% 	375,190 	DE
District of Columbia 	3 	21,256 	9.34% 	– 	202,970 	89.18% 	3 	1,485 	0.65% 	– 	502 	0.22% 	– 	0 	0.00% 	– 	737 	0.32% 	– 	636 	0.28% 	– 	−181,714 	−79.84% 	227,586 	DC
Florida 	27 	3,964,522 	52.10% 	27 	3,583,544 	47.09% 	– 	32,971 	0.43% 	– 	11,996 	0.16% 	– 	6,626 	0.09% 	– 	3,917 	0.05% 	– 	6,234 	0.08% 	– 	380,978 	5.01% 	7,609,810 	FL
Georgia 	15 	1,914,254 	57.97% 	15 	1,366,149 	41.37% 	– 	2,231 	0.07% 	– 	18,387 	0.56% 	– 	580 	0.02% 	– 	228 	0.01% 	– 	46 	0.00% 	– 	548,105 	16.60% 	3,301,875 	GA
Hawaii 	4 	194,191 	45.26% 	– 	231,708 	54.01% 	4 	0 	0.00% 	– 	1,377 	0.32% 	– 	0 	0.00% 	– 	1,737 	0.40% 	– 	0 	0.00% 	– 	−37,517 	−8.74% 	429,013 	HI
Idaho 	4 	409,235 	68.38% 	4 	181,098 	30.26% 	– 	1,115 	0.19% 	– 	3,844 	0.64% 	– 	3,084 	0.52% 	– 	58 	0.01% 	– 	13 	0.00% 	– 	228,137 	38.12% 	598,447 	ID
Illinois 	21 	2,345,946 	44.48% 	– 	2,891,550 	54.82% 	21 	3,571 	0.07% 	– 	32,442 	0.62% 	– 	440 	0.01% 	– 	241 	0.00% 	– 	132 	0.00% 	– 	−545,604 	−10.34% 	5,274,322 	IL
Indiana 	11 	1,479,438 	59.94% 	11 	969,011 	39.26% 	– 	1,328 	0.05% 	– 	18,058 	0.73% 	– 	0 	0.00% 	– 	102 	0.00% 	– 	65 	0.00% 	– 	510,427 	20.68% 	2,468,002 	IN
Iowa 	7 	751,957 	49.90% 	7 	741,898 	49.23% 	– 	5,973 	0.40% 	– 	2,992 	0.20% 	– 	1,304 	0.09% 	– 	1,141 	0.08% 	– 	1,643 	0.11% 	– 	10,059 	0.67% 	1,506,908 	IA
Kansas 	6 	736,456 	62.00% 	6 	434,993 	36.62% 	– 	9,348 	0.79% 	– 	4,013 	0.34% 	– 	2,899 	0.24% 	– 	33 	0.00% 	– 	14 	0.00% 	– 	301,463 	25.38% 	1,187,756 	KS
Kentucky 	8 	1,069,439 	59.55% 	8 	712,733 	39.69% 	– 	8,856 	0.49% 	– 	2,619 	0.15% 	– 	2,213 	0.12% 	– 	0 	0.00% 	– 	22 	0.00% 	– 	356,706 	19.86% 	1,795,882 	KY
Louisiana 	9 	1,102,169 	56.72% 	9 	820,299 	42.22% 	– 	7,032 	0.36% 	– 	2,781 	0.14% 	– 	5,203 	0.27% 	– 	1,276 	0.07% 	– 	4,346 	0.22% 	– 	281,870 	14.51% 	1,943,106 	LA
Maine★ 	4 	330,201 	44.58% 	– 	396,842 	53.57% 	4 	8,069 	1.09% 	– 	1,965 	0.27% 	– 	735 	0.10% 	– 	2,936 	0.40% 	– 	4 	0.00% 	– 	−66,641 	−9.00% 	740,752 	ME
Maryland 	10 	1,024,703 	42.93% 	– 	1,334,493 	55.91% 	10 	11,854 	0.50% 	– 	6,094 	0.26% 	– 	3,421 	0.14% 	– 	3,632 	0.15% 	– 	2,481 	0.10% 	– 	−309,790 	−12.98% 	2,386,678 	MD
Massachusetts 	12 	1,071,109 	36.78% 	– 	1,803,800 	61.94% 	12 	4,806 	0.17% 	– 	15,022 	0.52% 	– 	0 	0.00% 	– 	10,623 	0.36% 	– 	7,028 	0.24% 	– 	−732,691 	−25.16% 	2,912,388 	MA
Michigan 	17 	2,313,746 	47.81% 	– 	2,479,183 	51.23% 	17 	24,035 	0.50% 	– 	10,552 	0.22% 	– 	4,980 	0.10% 	– 	5,325 	0.11% 	– 	1,431 	0.03% 	– 	−165,437 	−3.42% 	4,839,252 	MI
Minnesota 	10 	1,346,695 	47.61% 	– 	1,445,014 	51.09% 	9 	18,683 	0.66% 	– 	4,639 	0.16% 	– 	3,074 	0.11% 	– 	4,408 	0.16% 	– 	5,874 	0.21% 	– 	−98,319 	−3.48% 	2,828,387 	MN
Mississippi 	6 	684,981 	59.45% 	6 	458,094 	39.76% 	– 	3,177 	0.28% 	– 	1,793 	0.16% 	– 	1,759 	0.15% 	– 	1,073 	0.09% 	– 	1,268 	0.11% 	– 	226,887 	19.69% 	1,152,145 	MS
Missouri 	11 	1,455,713 	53.30% 	11 	1,259,171 	46.10% 	– 	1,294 	0.05% 	– 	9,831 	0.36% 	– 	5,355 	0.20% 	– 	0 	0.00% 	– 	0 	0.00% 	– 	196,542 	7.20% 	2,731,364 	MO
Montana 	3 	266,063 	59.07% 	3 	173,710 	38.56% 	– 	6,168 	1.37% 	– 	1,733 	0.38% 	– 	1,764 	0.39% 	– 	996 	0.22% 	– 	11 	0.00% 	– 	92,353 	20.50% 	450,445 	MT
Nebraska★ 	5 	512,814 	65.90% 	5 	254,328 	32.68% 	– 	5,698 	0.73% 	– 	2,041 	0.26% 	– 	1,314 	0.17% 	– 	978 	0.13% 	– 	1,013 	0.13% 	– 	258,486 	33.22% 	778,186 	NE
Nevada 	5 	418,690 	50.47% 	5 	397,190 	47.88% 	– 	4,838 	0.58% 	– 	3,176 	0.38% 	– 	1,152 	0.14% 	– 	853 	0.10% 	– 	3,688 	0.44% 	– 	21,500 	2.59% 	829,587 	NV
New Hampshire 	4 	331,237 	48.87% 	– 	340,511 	50.24% 	4 	4,479 	0.66% 	– 	372 	0.05% 	– 	161 	0.02% 	– 	0 	0.00% 	– 	978 	0.14% 	– 	−9,274 	−1.37% 	677,738 	NH
New Jersey 	15 	1,670,003 	46.24% 	– 	1,911,430 	52.92% 	15 	19,418 	0.54% 	– 	4,514 	0.12% 	– 	2,750 	0.08% 	– 	1,807 	0.05% 	– 	1,769 	0.05% 	– 	−241,427 	−6.68% 	3,611,691 	NJ
New Mexico 	5 	376,930 	49.84% 	5 	370,942 	49.05% 	– 	4,053 	0.54% 	– 	2,382 	0.31% 	– 	771 	0.10% 	– 	1,226 	0.16% 	– 	0 	0.00% 	– 	5,988 	0.79% 	756,304 	NM
New York 	31 	2,962,567 	40.08% 	– 	4,314,280 	58.37% 	31 	99,873 	1.35% 	– 	11,607 	0.16% 	– 	207 	0.00% 	– 	87 	0.00% 	– 	2,415 	0.03% 	– 	−1,351,713 	−18.29% 	7,391,036 	NY
North Carolina 	15 	1,961,166 	56.02% 	15 	1,525,849 	43.58% 	– 	1,805 	0.05% 	– 	11,731 	0.34% 	– 	0 	0.00% 	– 	108 	0.00% 	– 	348 	0.01% 	– 	435,317 	12.43% 	3,501,007 	NC
North Dakota 	3 	196,651 	62.86% 	3 	111,052 	35.50% 	– 	3,756 	1.20% 	– 	851 	0.27% 	– 	514 	0.16% 	– 	0 	0.00% 	– 	9 	0.00% 	– 	85,599 	27.36% 	312,833 	ND
Ohio 	20 	2,859,768 	50.81% 	20 	2,741,167 	48.71% 	– 	0 	0.00% 	– 	14,676 	0.26% 	– 	11,939 	0.21% 	– 	192 	0.00% 	– 	166 	0.00% 	– 	118,601 	2.11% 	5,627,908 	OH
Oklahoma 	7 	959,792 	65.57% 	7 	503,966 	34.43% 	– 	0 	0.00% 	– 	0 	0.00% 	– 	0 	0.00% 	– 	0 	0.00% 	– 	0 	0.00% 	– 	455,826 	31.14% 	1,463,758 	OK
Oregon 	7 	866,831 	47.19% 	– 	943,163 	51.35% 	7 	0 	0.00% 	– 	7,260 	0.40% 	– 	5,257 	0.29% 	– 	5,315 	0.29% 	– 	8,956 	0.49% 	– 	−76,332 	−4.16% 	1,836,782 	OR
Pennsylvania 	21 	2,793,847 	48.42% 	– 	2,938,095 	50.92% 	21 	2,656 	0.05% 	– 	21,185 	0.37% 	– 	6,318 	0.11% 	– 	6,319 	0.11% 	– 	1,170 	0.02% 	– 	−144,248 	−2.50% 	5,769,590 	PA
Rhode Island 	4 	169,046 	38.67% 	– 	259,765 	59.42% 	4 	4,651 	1.06% 	– 	907 	0.21% 	– 	339 	0.08% 	– 	1,333 	0.30% 	– 	1,093 	0.25% 	– 	−90,719 	−20.75% 	437,134 	RI
South Carolina 	8 	937,974 	57.98% 	8 	661,699 	40.90% 	– 	5,520 	0.34% 	– 	3,608 	0.22% 	– 	5,317 	0.33% 	– 	1,488 	0.09% 	– 	2,124 	0.13% 	– 	276,275 	17.08% 	1,617,730 	SC
South Dakota 	3 	232,584 	59.91% 	3 	149,244 	38.44% 	– 	4,320 	1.11% 	– 	964 	0.25% 	– 	1,103 	0.28% 	– 	0 	0.00% 	– 	0 	0.00% 	– 	83,340 	21.47% 	388,215 	SD
Tennessee 	11 	1,384,375 	56.80% 	11 	1,036,477 	42.53% 	– 	8,992 	0.37% 	– 	4,866 	0.20% 	– 	2,570 	0.11% 	– 	33 	0.00% 	– 	6 	0.00% 	– 	347,898 	14.27% 	2,437,319 	TN
Texas 	34 	4,526,917 	61.09% 	34 	2,832,704 	38.22% 	– 	9,159 	0.12% 	– 	38,787 	0.52% 	– 	1,636 	0.02% 	– 	1,014 	0.01% 	– 	548 	0.01% 	– 	1,694,213 	22.86% 	7,410,765 	TX
Utah 	5 	663,742 	71.54% 	5 	241,199 	26.00% 	– 	11,305 	1.22% 	– 	3,375 	0.36% 	– 	6,841 	0.74% 	– 	39 	0.00% 	– 	1,343 	0.14% 	– 	422,543 	45.54% 	927,844 	UT
Vermont 	3 	121,180 	38.80% 	– 	184,067 	58.94% 	3 	4,494 	1.44% 	– 	1,102 	0.35% 	– 	0 	0.00% 	– 	0 	0.00% 	– 	1,466 	0.47% 	– 	−62,887 	−20.14% 	312,309 	VT
Virginia 	13 	1,716,959 	53.68% 	13 	1,454,742 	45.48% 	– 	2,393 	0.07% 	– 	11,032 	0.34% 	– 	10,161 	0.32% 	– 	104 	0.00% 	– 	2,976 	0.09% 	– 	262,217 	8.20% 	3,198,367 	VA
Washington 	11 	1,304,894 	45.64% 	– 	1,510,201 	52.82% 	11 	23,283 	0.81% 	– 	11,955 	0.42% 	– 	3,922 	0.14% 	– 	2,974 	0.10% 	– 	1,855 	0.06% 	– 	−205,307 	−7.18% 	2,859,084 	WA
West Virginia 	5 	423,778 	56.06% 	5 	326,541 	43.20% 	– 	4,063 	0.54% 	– 	1,405 	0.19% 	– 	82 	0.01% 	– 	5 	0.00% 	– 	13 	0.00% 	– 	97,237 	12.86% 	755,887 	WV
Wisconsin 	10 	1,478,120 	49.32% 	– 	1,489,504 	49.70% 	10 	16,390 	0.55% 	– 	6,464 	0.22% 	– 	0 	0.00% 	– 	2,661 	0.09% 	– 	3,868 	0.13% 	– 	−11,384 	−0.38% 	2,997,007 	WI
Wyoming 	3 	167,629 	68.86% 	3 	70,776 	29.07% 	– 	2,741 	1.13% 	– 	1,171 	0.48% 	– 	631 	0.26% 	– 	0 	0.00% 	– 	480 	0.20% 	– 	96,853 	39.79% 	243,428 	WY""",
    """Alabama 	9 	941,173 	56.48% 	9 	692,611 	41.57% 	– 	18,323 	1.10% 	– 	6,351 	0.38% 	– 	5,893 	0.35% 	– 	775 	0.05% 	– 	447 	0.03% 	– 	699 	0.04% 	– 	248,562 	14.92% 	1,666,272 	AL
Alaska 	3 	167,398 	58.62% 	3 	79,004 	27.67% 	– 	28,747 	10.07% 	– 	5,192 	1.82% 	– 	2,636 	0.92% 	– 	596 	0.21% 	– 	919 	0.32% 	– 	1,068 	0.37% 	– 	88,394 	30.95% 	285,560 	AK
Arizona★ 	8 	781,652 	51.02% 	8 	685,341 	44.73% 	– 	45,645 	2.98% 	– 	12,373 	0.81% 	– 	– 	– 	– 	110 	0.01% 	– 	1,120 	0.07% 	– 	5,775 	0.38% 	– 	96,311 	6.29% 	1,532,016 	AZ
Arkansas 	6 	472,940 	51.31% 	6 	422,768 	45.86% 	– 	13,421 	1.46% 	– 	7,358 	0.80% 	– 	2,781 	0.30% 	– 	1,415 	0.15% 	– 	1,098 	0.12% 	– 	– 	– 	– 	50,172 	5.44% 	921,781 	AR
California 	54 	4,567,429 	41.65% 	– 	5,861,203 	53.45% 	54 	418,707 	3.82% 	– 	44,987 	0.41% 	– 	45,520 	0.42% 	– 	17,042 	0.16% 	– 	10,934 	0.10% 	– 	34 	0.00% 	– 	−1,293,774 	−11.80% 	10,965,856 	CA
Colorado 	8 	883,748 	50.75% 	8 	738,227 	42.39% 	– 	91,434 	5.25% 	– 	10,465 	0.60% 	– 	12,799 	0.73% 	– 	1,319 	0.08% 	– 	2,240 	0.13% 	– 	1,136 	0.07% 	– 	145,521 	8.36% 	1,741,368 	CO
Connecticut 	8 	561,094 	38.44% 	– 	816,015 	55.91% 	8 	64,452 	4.42% 	– 	4,731 	0.32% 	– 	3,484 	0.24% 	– 	9,695 	0.66% 	– 	40 	0.00% 	– 	14 	0.00% 	– 	−254,921 	−17.47% 	1,459,525 	CT
Delaware 	3 	137,288 	41.90% 	– 	180,068 	54.96% 	3 	8,307 	2.54% 	– 	777 	0.24% 	– 	774 	0.24% 	– 	208 	0.06% 	– 	107 	0.03% 	– 	93 	0.03% 	– 	−42,780 	−13.06% 	327,622 	DE
D.C. 	3 	18,073 	8.95% 	– 	171,923 	85.16% 	3 	10,576 	5.24% 	– 	– 	– 	– 	669 	0.33% 	– 	– 	– 	– 	– 	– 	– 	653 	0.32% 	1 	−153,850 	−76.20% 	201,894 	DC
Florida 	25 	2,912,790 	48.85% 	25 	2,912,253 	48.84% 	– 	97,488 	1.63% 	– 	17,484 	0.29% 	– 	16,415 	0.28% 	– 	1,371 	0.02% 	– 	2,281 	0.04% 	– 	3,028 	0.05% 	– 	537 	0.01% 	5,963,110 	FL
Georgia 	13 	1,419,720 	54.67% 	13 	1,116,230 	42.98% 	– 	13,432 	0.52% 	– 	10,926 	0.42% 	– 	36,332 	1.40% 	– 	140 	0.01% 	– 	– 	– 	– 	24 	0.00% 	– 	303,490 	11.69% 	2,596,804 	GA
Hawaii 	4 	137,845 	37.46% 	– 	205,286 	55.79% 	4 	21,623 	5.88% 	– 	1,071 	0.29% 	– 	1,477 	0.40% 	– 	343 	0.09% 	– 	306 	0.08% 	– 	– 	– 	– 	−67,441 	−18.33% 	367,951 	HI
Idaho 	4 	336,937 	67.17% 	4 	138,637 	27.64% 	– 	12,292 	2.45% 	– 	7,615 	1.52% 	– 	3,488 	0.70% 	– 	1,469 	0.29% 	– 	1,177 	0.23% 	– 	6 	0.00% 	– 	198,300 	39.53% 	501,621 	ID
Illinois 	22 	2,019,421 	42.58% 	– 	2,589,026 	54.60% 	22 	103,759 	2.19% 	– 	16,106 	0.34% 	– 	11,623 	0.25% 	– 	57 	0.00% 	– 	2,127 	0.04% 	– 	4 	0.00% 	– 	−569,605 	−12.01% 	4,742,123 	IL
Indiana 	12 	1,245,836 	56.65% 	12 	901,980 	41.01% 	– 	18,531 	0.84% 	– 	16,959 	0.77% 	– 	15,530 	0.71% 	– 	200 	0.01% 	– 	167 	0.01% 	– 	99 	0.00% 	– 	343,856 	15.63% 	2,199,302 	IN
Iowa 	7 	634,373 	48.22% 	– 	638,517 	48.54% 	7 	29,374 	2.23% 	– 	5,731 	0.44% 	– 	3,209 	0.24% 	– 	613 	0.05% 	– 	2,281 	0.17% 	– 	1,465 	0.11% 	– 	−4,144 	−0.31% 	1,315,563 	IA
Kansas 	6 	622,332 	58.04% 	6 	399,276 	37.24% 	– 	36,086 	3.37% 	– 	7,370 	0.69% 	– 	4,525 	0.42% 	– 	1,254 	0.12% 	– 	1,375 	0.13% 	– 	– 	– 	– 	223,056 	20.80% 	1,072,218 	KS
Kentucky 	8 	872,492 	56.50% 	8 	638,898 	41.37% 	– 	23,192 	1.50% 	– 	4,173 	0.27% 	– 	2,896 	0.19% 	– 	923 	0.06% 	– 	1,533 	0.10% 	– 	80 	0.01% 	– 	233,594 	15.13% 	1,544,187 	KY
Louisiana 	9 	927,871 	52.55% 	9 	792,344 	44.88% 	– 	20,473 	1.16% 	– 	14,356 	0.81% 	– 	2,951 	0.17% 	– 	5,483 	0.31% 	– 	1,075 	0.06% 	– 	1,103 	0.06% 	– 	135,527 	7.68% 	1,765,656 	LA
Maine† 	4 	286,616 	43.97% 	– 	319,951 	49.09% 	4 	37,127 	5.70% 	– 	4,443 	0.68% 	– 	3,074 	0.47% 	– 	579 	0.09% 	– 	– 	– 	– 	27 	0.00% 	– 	−33,335 	−5.11% 	651,817 	ME
Maryland 	10 	813,797 	40.18% 	– 	1,145,782 	56.57% 	10 	53,768 	2.65% 	– 	4,248 	0.21% 	– 	5,310 	0.26% 	– 	919 	0.05% 	– 	176 	0.01% 	– 	1,480 	0.07% 	– 	−331,985 	−16.39% 	2,025,480 	MD
Massachusetts 	12 	878,502 	32.50% 	– 	1,616,487 	59.80% 	12 	173,564 	6.42% 	– 	11,149 	0.41% 	– 	16,366 	0.61% 	– 	– 	– 	– 	2,884 	0.11% 	– 	4,032 	0.15% 	– 	−737,985 	−27.30% 	2,702,984 	MA
Michigan 	18 	1,953,139 	46.15% 	– 	2,170,418 	51.28% 	18 	84,165 	1.99% 	– 	1,851 	0.04% 	– 	16,711 	0.39% 	– 	3,791 	0.09% 	– 	2,426 	0.06% 	– 	– 	– 	– 	−217,279 	−5.13% 	4,232,501 	MI
Minnesota 	10 	1,109,659 	45.50% 	– 	1,168,266 	47.91% 	10 	126,696 	5.20% 	– 	22,166 	0.91% 	– 	5,282 	0.22% 	– 	3,272 	0.13% 	– 	2,294 	0.09% 	– 	1,050 	0.04% 	– 	−58,607 	−2.40% 	2,438,685 	MN
Mississippi 	7 	572,844 	57.62% 	7 	404,614 	40.70% 	– 	8,122 	0.82% 	– 	2,265 	0.23% 	– 	2,009 	0.20% 	– 	3,267 	0.33% 	– 	450 	0.05% 	– 	613 	0.06% 	– 	168,230 	16.92% 	994,184 	MS
Missouri 	11 	1,189,924 	50.42% 	11 	1,111,138 	47.08% 	– 	38,515 	1.63% 	– 	9,818 	0.42% 	– 	7,436 	0.32% 	– 	1,957 	0.08% 	– 	1,104 	0.05% 	– 	– 	– 	– 	78,786 	3.34% 	2,359,892 	MO
Montana 	3 	240,178 	58.44% 	3 	137,126 	33.36% 	– 	24,437 	5.95% 	– 	5,697 	1.39% 	– 	1,718 	0.42% 	– 	1,155 	0.28% 	– 	675 	0.16% 	– 	11 	0.00% 	– 	103,052 	25.07% 	410,997 	MT
Nebraska† 	5 	433,862 	62.25% 	5 	231,780 	33.25% 	– 	24,540 	3.52% 	– 	3,646 	0.52% 	– 	2,245 	0.32% 	– 	468 	0.07% 	– 	478 	0.07% 	– 	– 	– 	– 	202,082 	28.99% 	697,019 	NE
Nevada 	4 	301,575 	49.52% 	4 	279,978 	45.98% 	– 	15,008 	2.46% 	– 	4,747 	0.78% 	– 	3,311 	0.54% 	– 	621 	0.10% 	– 	415 	0.07% 	– 	3,315 	0.54% 	– 	21,597 	3.55% 	608,970 	NV
New Hampshire 	4 	273,559 	48.07% 	4 	266,348 	46.80% 	– 	22,198 	3.90% 	– 	2,615 	0.46% 	– 	2,757 	0.48% 	– 	328 	0.06% 	– 	55 	0.01% 	– 	1,221 	0.21% 	– 	7,211 	1.27% 	569,081 	NH
New Jersey 	15 	1,284,173 	40.29% 	– 	1,788,850 	56.13% 	15 	94,554 	2.97% 	– 	6,989 	0.22% 	– 	6,312 	0.20% 	– 	1,409 	0.04% 	– 	2,215 	0.07% 	– 	2,724 	0.09% 	– 	−504,677 	−15.83% 	3,187,226 	NJ
New Mexico 	5 	286,417 	47.85% 	– 	286,783 	47.91% 	5 	21,251 	3.55% 	– 	1,392 	0.23% 	– 	2,058 	0.34% 	– 	343 	0.06% 	– 	361 	0.06% 	– 	– 	– 	– 	−366 	−0.06% 	598,605 	NM
New York 	33 	2,403,374 	35.23% 	– 	4,107,697 	60.21% 	33 	244,030 	3.58% 	– 	31,599 	0.46% 	– 	7,649 	0.11% 	– 	1,498 	0.02% 	– 	24,361 	0.36% 	– 	1,791 	0.03% 	– 	−1,704,323 	−24.98% 	6,821,999 	NY
North Carolina 	14 	1,631,163 	56.03% 	14 	1,257,692 	43.20% 	– 	– 	– 	– 	8,874 	0.30% 	– 	12,307 	0.42% 	– 	– 	– 	– 	– 	– 	– 	1,226 	0.04% 	– 	373,471 	12.83% 	2,911,262 	NC
North Dakota 	3 	174,852 	60.66% 	3 	95,284 	33.06% 	– 	9,486 	3.29% 	– 	7,288 	2.53% 	– 	660 	0.23% 	– 	373 	0.13% 	– 	313 	0.11% 	– 	– 	– 	– 	79,568 	27.60% 	288,256 	ND
Ohio 	21 	2,351,209 	49.97% 	21 	2,186,190 	46.46% 	– 	117,857 	2.50% 	– 	26,724 	0.57% 	– 	13,475 	0.29% 	– 	3,823 	0.08% 	– 	6,169 	0.13% 	– 	10 	0.00% 	– 	165,019 	3.51% 	4,705,457 	OH
Oklahoma 	8 	744,337 	60.31% 	8 	474,276 	38.43% 	– 	– 	– 	– 	9,014 	0.73% 	– 	6,602 	0.53% 	– 	– 	– 	– 	– 	– 	– 	– 	– 	– 	270,061 	21.88% 	1,234,229 	OK
Oregon 	7 	713,577 	46.52% 	– 	720,342 	46.96% 	7 	77,357 	5.04% 	– 	7,063 	0.46% 	– 	7,447 	0.49% 	– 	2,189 	0.14% 	– 	2,574 	0.17% 	– 	3,419 	0.22% 	– 	−6,765 	−0.44% 	1,533,968 	OR
Pennsylvania 	23 	2,281,127 	46.43% 	– 	2,485,967 	50.60% 	23 	103,392 	2.10% 	– 	16,023 	0.33% 	– 	11,248 	0.23% 	– 	14,428 	0.29% 	– 	– 	– 	– 	934 	0.02% 	– 	−204,840 	−4.17% 	4,913,119 	PA
Rhode Island 	4 	130,555 	31.91% 	– 	249,508 	60.99% 	4 	25,052 	6.12% 	– 	2,273 	0.56% 	– 	742 	0.18% 	– 	97 	0.02% 	– 	271 	0.07% 	– 	614 	0.15% 	– 	−118,953 	−29.08% 	409,112 	RI
South Carolina 	8 	785,937 	56.84% 	8 	565,561 	40.90% 	– 	20,200 	1.46% 	– 	3,519 	0.25% 	– 	4,876 	0.35% 	– 	1,682 	0.12% 	– 	942 	0.07% 	– 	– 	– 	– 	220,376 	15.94% 	1,382,717 	SC
South Dakota 	3 	190,700 	60.30% 	3 	118,804 	37.56% 	– 	– 	– 	– 	3,322 	1.05% 	– 	1,662 	0.53% 	– 	1,781 	0.56% 	– 	– 	– 	– 	– 	– 	– 	71,896 	22.73% 	316,269 	SD
Tennessee 	11 	1,061,949 	51.15% 	11 	981,720 	47.28% 	– 	19,781 	0.95% 	– 	4,250 	0.20% 	– 	4,284 	0.21% 	– 	1,015 	0.05% 	– 	613 	0.03% 	– 	2,569 	0.12% 	– 	80,229 	3.86% 	2,076,181 	TN
Texas 	32 	3,799,639 	59.30% 	32 	2,433,746 	37.98% 	– 	137,994 	2.15% 	– 	12,394 	0.19% 	– 	23,160 	0.36% 	– 	567 	0.01% 	– 	– 	– 	– 	137 	0.00% 	– 	1,365,893 	21.32% 	6,407,637 	TX
Utah 	5 	515,096 	66.83% 	5 	203,053 	26.34% 	– 	35,850 	4.65% 	– 	9,319 	1.21% 	– 	3,616 	0.47% 	– 	2,709 	0.35% 	– 	763 	0.10% 	– 	348 	0.05% 	– 	312,043 	40.49% 	770,754 	UT
Vermont 	3 	119,775 	40.70% 	– 	149,022 	50.63% 	3 	20,374 	6.92% 	– 	2,192 	0.74% 	– 	784 	0.27% 	– 	153 	0.05% 	– 	219 	0.07% 	– 	1,789 	0.61% 	– 	−29,247 	−9.94% 	294,308 	VT
Virginia 	13 	1,437,490 	52.47% 	13 	1,217,290 	44.44% 	– 	59,398 	2.17% 	– 	5,455 	0.20% 	– 	15,198 	0.55% 	– 	1,809 	0.07% 	– 	171 	0.01% 	– 	2,636 	0.10% 	– 	220,200 	8.04% 	2,739,447 	VA
Washington 	11 	1,108,864 	44.58% 	– 	1,247,652 	50.16% 	11 	103,002 	4.14% 	– 	7,171 	0.29% 	– 	13,135 	0.53% 	– 	1,989 	0.08% 	– 	2,927 	0.12% 	– 	2,693 	0.11% 	– 	−138,788 	−5.58% 	2,487,433 	WA
West Virginia 	5 	336,475 	51.92% 	5 	295,497 	45.59% 	– 	10,680 	1.65% 	– 	3,169 	0.49% 	– 	1,912 	0.30% 	– 	23 	0.00% 	– 	367 	0.06% 	– 	1 	0.00% 	– 	40,978 	6.32% 	648,124 	WV
Wisconsin 	11 	1,237,279 	47.61% 	– 	1,242,987 	47.83% 	11 	94,070 	3.62% 	– 	11,471 	0.44% 	– 	6,640 	0.26% 	– 	2,042 	0.08% 	– 	853 	0.03% 	– 	3,265 	0.13% 	– 	−5,708 	−0.22% 	2,598,607 	WI
Wyoming 	3 	147,947 	67.76% 	3 	60,481 	27.70% 	– 	4,625 	2.12% 	– 	2,724 	1.25% 	– 	1,443 	0.66% 	– 	720 	0.33% 	– 	411 	0.19% 	– 	– 	– 	– 	87,466 	40.06% 	218,351 	WY""",
    """Alabama 	9 	662,165 	43.16% 	– 	769,044 	50.12% 	9 	92,149 	6.01% 	– 	– 	– 	– 	5,290 	0.34% 	– 	5,701 	0.37% 	– 	−106,879 	−6.97% 	1,534,349 	AL
Alaska 	3 	80,380 	33.27% 	– 	122,746 	50.80% 	3 	26,333 	10.90% 	– 	7,597 	3.14% 	– 	2,276 	0.94% 	– 	2,288 	0.95% 	– 	−42,366 	−17.53% 	241,620 	AK
Arizona 	8 	653,288 	46.52% 	8 	622,073 	44.29% 	– 	112,072 	7.98% 	– 	2,062 	0.15% 	– 	14,358 	1.02% 	– 	552 	0.04% 	– 	31,215 	2.22% 	1,404,405 	AZ
Arkansas 	6 	475,171 	53.74% 	6 	325,416 	36.80% 	– 	69,884 	7.90% 	– 	3,649 	0.41% 	– 	3,076 	0.35% 	– 	7,066 	0.80% 	– 	149,755 	16.94% 	884,262 	AR
California 	54 	5,119,835 	51.10% 	54 	3,828,380 	38.21% 	– 	697,847 	6.96% 	– 	237,016 	2.37% 	– 	73,600 	0.73% 	– 	62,806 	0.63% 	– 	1,291,455 	12.89% 	10,019,484 	CA
Colorado 	8 	671,152 	44.43% 	– 	691,848 	45.80% 	8 	99,629 	6.59% 	– 	25,070 	1.66% 	– 	12,392 	0.82% 	– 	10,613 	0.70% 	– 	−20,696 	−1.37% 	1,510,704 	CO
Connecticut 	8 	735,740 	52.83% 	8 	483,109 	34.69% 	– 	139,523 	10.02% 	– 	24,321 	1.75% 	– 	5,788 	0.42% 	– 	4,133 	0.30% 	– 	252,631 	18.14% 	1,392,614 	CT
Delaware 	3 	140,355 	51.80% 	3 	99,062 	36.58% 	– 	28,719 	10.60% 	– 	18 	0.01% 	– 	2,052 	0.76% 	– 	639 	0.24% 	– 	41,293 	15.25% 	270,845 	DE
D.C. 	3 	158,220 	85.19% 	3 	17,339 	9.34% 	– 	3,611 	1.94% 	– 	4,780 	2.57% 	– 	588 	0.32% 	– 	1,188 	0.64% 	– 	140,881 	75.85% 	185,726 	DC
Florida 	25 	2,546,870 	48.02% 	25 	2,244,536 	42.32% 	– 	483,870 	9.12% 	– 	4,101 	0.08% 	– 	23,965 	0.45% 	– 	452 	0.01% 	– 	302,334 	5.70% 	5,303,794 	FL
Georgia 	13 	1,053,849 	45.84% 	– 	1,080,843 	47.01% 	13 	146,337 	6.37% 	– 	– 	– 	– 	17,870 	0.78% 	– 	172 	0.01% 	– 	−26,994 	−1.17% 	2,299,071 	GA
Hawaii 	4 	205,012 	56.93% 	4 	113,943 	31.64% 	– 	27,358 	7.60% 	– 	10,386 	2.88% 	– 	2,493 	0.69% 	– 	928 	0.26% 	– 	91,069 	25.29% 	360,120 	HI
Idaho 	4 	165,443 	33.65% 	– 	256,595 	52.18% 	4 	62,518 	12.71% 	– 	– 	– 	– 	3,325 	0.68% 	– 	3,838 	0.78% 	– 	−91,152 	−18.54% 	491,719 	ID
Illinois 	22 	2,341,744 	54.32% 	22 	1,587,021 	36.81% 	– 	346,408 	8.03% 	– 	1,447 	0.03% 	– 	22,548 	0.52% 	– 	12,223 	0.29% 	– 	754,723 	17.51% 	4,311,391 	IL
Indiana 	12 	887,424 	41.55% 	– 	1,006,693 	47.13% 	12 	224,299 	10.50% 	– 	1,121 	0.05% 	– 	15,632 	0.73% 	– 	673 	0.03% 	– 	−119,269 	−5.58% 	2,135,842 	IN
Iowa 	7 	620,258 	50.26% 	7 	492,644 	39.92% 	– 	105,159 	8.52% 	– 	6,550 	0.53% 	– 	2,315 	0.19% 	– 	7,149 	0.58% 	– 	127,614 	10.34% 	1,234,075 	IA
Kansas 	6 	387,659 	36.08% 	– 	583,245 	54.29% 	6 	92,639 	8.62% 	– 	914 	0.09% 	– 	4,557 	0.42% 	– 	5,286 	0.49% 	– 	−195,586 	−18.21% 	1,074,300 	KS
Kentucky 	8 	636,614 	45.84% 	8 	623,283 	44.88% 	– 	120,396 	8.67% 	– 	701 	0.05% 	– 	4,009 	0.29% 	– 	3,705 	0.27% 	– 	13,331 	0.96% 	1,388,708 	KY
Louisiana 	9 	927,837 	52.01% 	9 	712,586 	39.94% 	– 	123,293 	6.91% 	– 	4,719 	0.26% 	– 	7,499 	0.42% 	– 	8,025 	0.45% 	– 	215,251 	12.07% 	1,783,959 	LA
Maine 	4 	312,788 	51.62% 	4 	186,378 	30.76% 	– 	85,970 	14.19% 	– 	15,279 	2.52% 	– 	2,996 	0.49% 	– 	2,486 	0.41% 	– 	126,410 	20.86% 	605,897 	ME
Maryland 	10 	966,207 	54.25% 	10 	681,530 	38.27% 	– 	115,812 	6.50% 	– 	2,606 	0.15% 	– 	8,765 	0.49% 	– 	5,950 	0.33% 	– 	284,677 	15.99% 	1,780,870 	MD
Massachusetts 	12 	1,571,763 	61.47% 	12 	718,107 	28.09% 	– 	227,217 	8.89% 	– 	4,734 	0.19% 	– 	20,426 	0.80% 	– 	14,538 	0.57% 	– 	853,656 	33.39% 	2,556,785 	MA
Michigan 	18 	1,989,653 	51.69% 	18 	1,481,212 	38.48% 	– 	336,670 	8.75% 	– 	2,322 	0.06% 	– 	27,670 	0.72% 	– 	11,317 	0.29% 	– 	508,441 	13.21% 	3,848,844 	MI
Minnesota 	10 	1,120,438 	51.10% 	10 	766,476 	34.96% 	– 	257,704 	11.75% 	– 	24,908 	1.14% 	– 	8,271 	0.38% 	– 	14,843 	0.68% 	– 	353,962 	16.14% 	2,192,640 	MN
Mississippi 	7 	394,022 	44.08% 	– 	439,838 	49.21% 	7 	52,222 	5.84% 	– 	– 	– 	– 	2,809 	0.31% 	– 	4,966 	0.56% 	– 	−45,816 	−5.13% 	893,857 	MS
Missouri 	11 	1,025,935 	47.54% 	11 	890,016 	41.24% 	– 	217,188 	10.06% 	– 	534 	0.02% 	– 	10,522 	0.49% 	– 	13,870 	0.64% 	– 	135,919 	6.30% 	2,158,065 	MO
Montana 	3 	167,922 	41.23% 	– 	179,652 	44.11% 	3 	55,229 	13.56% 	– 	– 	– 	– 	2,526 	0.62% 	– 	1,932 	0.47% 	– 	−11,730 	−2.88% 	407,261 	MT
Nebraska 	5 	236,761 	34.95% 	– 	363,467 	53.65% 	5 	71,278 	10.52% 	– 	– 	– 	– 	2,792 	0.41% 	– 	3,117 	0.46% 	– 	−126,706 	−18.70% 	677,415 	NE
Nevada 	4 	203,974 	43.93% 	4 	199,244 	42.91% 	– 	43,986 	9.47% 	– 	4,730 	1.02% 	– 	4,460 	0.96% 	– 	7,885 	1.70% 	– 	4,730 	1.02% 	464,279 	NV
New Hampshire 	4 	246,214 	49.32% 	4 	196,532 	39.37% 	– 	48,390 	9.69% 	– 	– 	– 	– 	4,237 	0.85% 	– 	3,802 	0.76% 	– 	49,682 	9.95% 	499,175 	NH
New Jersey 	15 	1,652,329 	53.72% 	15 	1,103,078 	35.86% 	– 	262,134 	8.52% 	– 	32,465 	1.06% 	– 	14,763 	0.48% 	– 	11,038 	0.36% 	– 	549,251 	17.86% 	3,075,807 	NJ
New Mexico 	5 	273,495 	49.18% 	5 	232,751 	41.86% 	– 	32,257 	5.80% 	– 	13,218 	2.38% 	– 	2,996 	0.54% 	– 	1,357 	0.24% 	– 	40,744 	7.33% 	556,074 	NM
New York 	33 	3,756,177 	59.47% 	33 	1,933,492 	30.61% 	– 	503,458 	7.97% 	– 	75,956 	1.20% 	– 	12,220 	0.19% 	– 	34,826 	0.55% 	– 	1,822,685 	28.86% 	6,316,129 	NY
North Carolina 	14 	1,107,849 	44.04% 	– 	1,225,938 	48.73% 	14 	168,059 	6.68% 	– 	2,108 	0.08% 	– 	8,740 	0.35% 	– 	3,113 	0.12% 	– 	−118,089 	−4.69% 	2,515,807 	NC
North Dakota 	3 	106,905 	40.13% 	– 	125,050 	46.94% 	3 	32,515 	12.20% 	– 	– 	– 	– 	847 	0.32% 	– 	1,094 	0.41% 	– 	−18,145 	−6.81% 	266,411 	ND
Ohio 	21 	2,148,222 	47.38% 	21 	1,859,883 	41.02% 	– 	483,207 	10.66% 	– 	2,962 	0.07% 	– 	12,851 	0.28% 	– 	27,309 	0.60% 	– 	288,339 	6.36% 	4,534,434 	OH
Oklahoma 	8 	488,105 	40.45% 	– 	582,315 	48.26% 	8 	130,788 	10.84% 	– 	– 	– 	– 	5,505 	0.46% 	– 	– 	– 	– 	−94,210 	−7.81% 	1,206,713 	OK
Oregon 	7 	649,641 	47.15% 	7 	538,152 	39.06% 	– 	121,221 	8.80% 	– 	49,415 	3.59% 	– 	8,903 	0.65% 	– 	10,428 	0.76% 	– 	111,489 	8.09% 	1,377,760 	OR
Pennsylvania 	23 	2,215,819 	49.17% 	23 	1,801,169 	39.97% 	– 	430,984 	9.56% 	– 	3,086 	0.07% 	– 	28,000 	0.62% 	– 	27,060 	0.60% 	– 	414,650 	9.20% 	4,506,118 	PA
Rhode Island 	4 	233,050 	59.71% 	4 	104,683 	26.82% 	– 	43,723 	11.20% 	– 	6,040 	1.55% 	– 	1,109 	0.28% 	– 	1,679 	0.43% 	– 	128,367 	32.89% 	390,284 	RI
South Carolina 	8 	504,051 	43.85% 	– 	573,458 	49.89% 	8 	64,386 	5.60% 	– 	– 	– 	– 	4,271 	0.37% 	– 	3,291 	0.29% 	– 	−69,407 	−6.04% 	1,149,457 	SC
South Dakota 	3 	139,333 	43.03% 	– 	150,543 	46.49% 	3 	31,250 	9.65% 	– 	– 	– 	– 	1,472 	0.45% 	– 	1,228 	0.38% 	– 	−11,210 	−3.46% 	323,826 	SD
Tennessee 	11 	909,146 	48.00% 	11 	863,530 	45.59% 	– 	105,918 	5.59% 	– 	6,427 	0.34% 	– 	5,020 	0.27% 	– 	4,064 	0.21% 	– 	45,616 	2.41% 	1,894,105 	TN
Texas 	32 	2,459,683 	43.83% 	– 	2,736,167 	48.76% 	32 	378,537 	6.75% 	– 	4,810 	0.09% 	– 	20,256 	0.36% 	– 	12,191 	0.22% 	– 	−276,484 	−4.93% 	5,611,644 	TX
Utah 	5 	221,633 	33.30% 	– 	361,911 	54.37% 	5 	66,461 	9.98% 	– 	4,615 	0.69% 	– 	4,129 	0.62% 	– 	6,880 	1.03% 	– 	−140,278 	−21.07% 	665,629 	UT
Vermont 	3 	137,894 	53.35% 	3 	80,352 	31.09% 	– 	31,024 	12.00% 	– 	5,585 	2.16% 	– 	1,183 	0.46% 	– 	2,411 	0.93% 	– 	57,542 	22.26% 	258,449 	VT
Virginia 	13 	1,091,060 	45.15% 	– 	1,138,350 	47.10% 	13 	159,861 	6.62% 	– 	– 	– 	– 	9,174 	0.38% 	– 	18,197 	0.75% 	– 	−47,290 	−1.96% 	2,416,642 	VA
Washington 	11 	1,123,323 	49.84% 	11 	840,712 	37.30% 	– 	201,003 	8.92% 	– 	60,322 	2.68% 	– 	12,522 	0.56% 	– 	15,955 	0.71% 	– 	282,611 	12.54% 	2,253,837 	WA
West Virginia 	5 	327,812 	51.51% 	5 	233,946 	36.76% 	– 	71,639 	11.26% 	– 	– 	– 	– 	3,062 	0.48% 	– 	– 	– 	– 	93,866 	14.75% 	636,459 	WV
Wisconsin 	11 	1,071,971 	48.81% 	11 	845,029 	38.48% 	– 	227,339 	10.35% 	– 	28,723 	1.31% 	– 	7,929 	0.36% 	– 	15,178 	0.69% 	– 	226,942 	10.33% 	2,196,169 	WI
Wyoming 	3 	77,934 	36.84% 	– 	105,388 	49.81% 	3 	25,928 	12.25% 	– 	– 	– 	– 	1,739 	0.82% 	– 	582 	0.28% 	– 	−27,454 	−12.98% 	211,571 	WY"""
]
ss = [a.split("\n") for a in ss]
ss = [[a.split("	") for a in l] for l in ss]
dem_ids = [2, 5, 5, 2]
rep_ids = [5, 2, 2, 5]
dem_rep = [[(a[dem_ids[i]], a[rep_ids[i]]) for a in l] for (i, l) in enumerate(ss)]
dem_rep = [[[int(re.sub("[^0-9]", "", x)) for x in a] for a in l] for l in dem_rep]
ps = [[x[0]/(x[0] + x[1]) for x in l] for l in dem_rep]
ps = torch.tensor(ps)
ps = torch.log(ps/(1-ps))
ps = torch.cat([mu.expand(1, 51), ps], dim=0)
h_mean = ps.mean(0)
sample_covariance = (1/(ps.shape[0] - 1)) * ((ps.unsqueeze(-1) - h_mean) * (ps.unsqueeze(-2) - h_mean)).sum(0)
prior_covariance = sample_covariance + 0.01 * torch.eye(51)
prior_scale_tril = prior_covariance.cholesky()
print(prior_scale_tril)

tensor([[ 0.1803,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0187,  0.1959,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0834,  0.0132,  0.1174,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [ 0.2349, -0.0444,  0.1021,  ...,  0.1120,  0.0000,  0.0000],
        [ 0.0321,  0.0727,  0.0165,  ...,  0.0027,  0.1014,  0.0000],
        [ 0.1619,  0.0744,  0.0800,  ...,  0.0134,  0.0037,  0.1064]])


We are now in a position to define our Bayesian model.

In [109]:
from contextlib import ExitStack

import pyro
import pyro.distributions as dist
from pyro.contrib.util import iter_plates_to_shape

def model(polling_allocation):
    with ExitStack() as stack:
        for plate in iter_plates_to_shape(polling_allocation.shape[:-1]):
            stack.enter_context(plate)
        theta = pyro.sample("theta", dist.MultivariateNormal(mu, scale_tril=prior_scale_tril))
        poll_results = pyro.sample("y", dist.Binomial(polling_allocation, logits=theta).to_event(1))
        dem_win_state = (theta > 0.).float()
        dem_electoral_college_votes = electoral_college_votes * dem_win_state
        dem_win = dem_electoral_college_votes.sum(-1) / electoral_college_votes.sum(-1) > .5
        pyro.sample("w", dist.Delta(dem_win))
        return poll_results, dem_win

In [118]:
simple_poll = 10 * torch.ones(100, 51)
poll_results, dem_win = model(simple_poll)

tensor([[ 0.,  0.,  3.,  ...,  0., 15.,  0.],
        [ 0.,  0.,  0.,  ...,  0., 15.,  0.],
        [ 0.,  0.,  3.,  ...,  6., 15.,  0.],
        ...,
        [ 0.,  0.,  0.,  ...,  0., 15.,  0.],
        [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  ...,  0., 15.,  0.]])
tensor([1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0,
        1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0,
        0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0,
        1, 1, 0, 0], dtype=torch.uint8)
tensor([390., 447., 445., 373., 643., 532., 614., 600., 926., 495., 495., 736.,
        350., 576., 471., 517., 384., 379., 395., 596., 637., 620., 581., 547.,
        455., 461., 429., 388., 556., 562., 564., 553., 632., 493., 402., 531.,
        324., 588., 511., 620., 466., 415., 390., 430., 238., 704., 525., 579.,
        345., 565., 318.]) ten